In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## Code forward pass

Note that the way scikit-learn encode the datasets is different (row represents a sample, columns features). So we need to change the matrix order in the multiplication


$\mathbf{x} = \begin{bmatrix}
x_0\\ 
x_1\\ 
x_2\\ 
...\\ 
x_d
\end{bmatrix}$ ,       $\mathbf{w} = \begin{bmatrix}
w_0\\ 
w_1\\ 
w_2\\ 
...\\ 
w_d
\end{bmatrix}$

$\large h(\mathbf{x}) = \mathbf{w}^T \mathbf{x}$

This formula assumes each exemple is represented by a column. However in Python (scikit-learn we do the oposite). 

Also the weights are represented by row vectors instead of columns (1D numpy arrays)

So for implemetation we need to change to:


$\large h(\mathbf{x}) = \mathbf{x}\mathbf{w}$


In [2]:
X = np.array([[1], [2], [3]])
y = np.array([1, 2, 3])
w = np.array([0])

In [3]:
def forward(X, W):
    return np.dot(X, W)
    #return np.matmul(X, W)
hx = forward(X, w)
hx

array([0, 0, 0])

## Code MSE

Equation:

$\Large J(\mathbf{w}) = \frac{1}{2N}\sum_{i=1}^{N} (h(\mathbf{x}_i) - y_i)^2$

4 steps:

- Get the difference between predictins and targets
- Square their differences
- Get the mean difference
- divide by two

Test case:

- hx = [0, 0, 0]

- y = [1, 2, 3]

Result

- J(w) = 2.333333...


In [4]:
def compute_MSE(hx, y):
    n_samples = y.size
    diff = (hx - y)
    diff_squared = diff ** 2
    sum_diff_squared = np.sum(diff_squared)
    MSE = sum_diff_squared/(2 * n_samples)        
    return MSE
compute_MSE(hx, y)

2.3333333333333335

## Code Gradients computation

$\large \nabla J(\mathbf{W}) = \frac{1}{N} \sum_{i=1}^{N} (h(\mathbf{x}_i) - y_i)\mathbf{x}_i$

Test case:

- hx = [0, 0, 0]

- X = [1, 2, 3]

- y = [1, 2, 3]

Result:

- J'(w) = -4.6666

In [5]:
def compute_gradient(X, hx, y):
        gradient = np.dot((hx - y), X)
        gradient = gradient/y.size
        return gradient
grad = compute_gradient(X, hx, y)
grad

array([-4.66666667])

## Code Gradient update

$\mathbf{w}^{iter+1} = \mathbf{w}^{iter} - \alpha \nabla J(\mathbf{w})$

In [6]:
alpha = 0.01
w = w - (alpha * grad)

## Régression Linéaire

In [7]:
class LinearRegression:
    
    def __init__(self, n_iter, learning_rate):
        self.n_iter = n_iter # for stopping criteria, we could do better here...
        self.learning_rate = learning_rate
        self.J_history = []
        
    def fit(self, X, y):
        """Function with the training code. Learn the parameters W of the model"""
        
        # keep information of number of examples and dimensions.
        self.n_samples = X.shape[0]
        self.n_features = X.shape[1]
        
        # add ones for the bias.
        x_0 = np.ones((self.n_samples, 1))
        X_tmp = np.hstack((x_0, X))
        
        # initialize weights 'W' of the model randomly
        self.W = np.random.rand(self.n_features + 1) # +1 include biais

        for _ in range(self.n_iter):
            # get prediction of current weights. (forward pass)
            hx = self.forward(X_tmp)
            
            # check the cost to know whether the model is learning or not.
            J = self.compute_cost(hx, y)
            self.J_history.append(J)
            
            # Gradient descent.
            gradient = self.compute_gradient(X_tmp, hx, y)
            self.W = self.W - (self.learning_rate * gradient) # update weights.
            
    def forward(self, X):
        hx = np.matmul(X, self.W)
        return hx
            
    def predict(self, X):
        """Our hypothesis h(x) to be applied over test data"""
        # add the ones vector.
        x_0 = np.ones((X.shape[0], 1))
        X_tmp = np.hstack((x_0, X))
        
        hx = self.forward(X_tmp)
        return hx
    
    def compute_cost(self, h_x, y):
        n_samples = y.size
        diff_squared = (h_x - y) ** 2
        sum_diff_squared = np.sum(diff_squared)
        MSE = sum_diff_squared/(2 * n_samples)
        return MSE
    
    def compute_gradient(self, X, h_x, y):
        gradient = np.dot((h_x - y), X)
        return gradient / y.size
    
    def plot_learning_curve(self):
        plt.plot(np.arange(self.n_iter), self.J_history)
        plt.xlabel('Iteration', fontsize=18)
        plt.ylabel('J(w)', fontsize=18)

In [12]:
## Logistic Regression implementation (Classification)
# Since it is pretty much the same as Linear, here I use inheritance to
# make things easier. Only thing that chages are:

# 1) addition of the logistic function after the linear combination
# 2) threshold for prediction (0.5)
class LogisticRegressionOurs(LinearRegression):
    
    def forward(self, X):
        z = super().forward(X)
        return 1/(1+np.exp(-z))

    def predict(self, X):
        y_pred = super().predict(X) >= 0.5
        return y_pred


In [13]:
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression as LRSklearn

logistic_sklearn = LRSklearn()
logistic_ours = LogisticRegressionOurs(1000, 0.1)
X, y = make_classification(n_samples=1000)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.25)

In [14]:
logistic_ours.fit(X_train, y_train)
logistic_sklearn.fit(X_train, y_train)

LogisticRegression()

In [15]:
from sklearn.metrics import accuracy_score

y_pred_ours = logistic_ours.predict(X_test)
y_pred_sklearn = logistic_sklearn.predict(X_test)
print(accuracy_score(y_pred_ours, y_test))
print(accuracy_score(y_pred_sklearn, y_test))

0.924
0.924
